# Imports and Agents Initialization

In [ ]:
import json

In [ ]:
# setup ollama to run llm locally
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# run ollama in background to pull a model
def run_ollama_in_background():
  import threading
  import subprocess
  import time

  def ollama_serve():
      subprocess.Popen(["ollama", "serve"])

  thread = threading.Thread(target=ollama_serve)
  thread.start()
  time.sleep(5)

run_ollama_in_background()

In [ ]:
# pull models
!ollama pull gemma3
# !ollama pull llava
# !ollama pull phi3

In [ ]:
# install python dependencies
!pip install ollama
!pip install groq
!pip install google-generativeai


In [ ]:
# --- LLM Model Functions ---

def groq_deepseek_r1(messages):
    #https://groq.com/
    from groq import Groq
    import re
    client = Groq(api_key="API-KEY")
    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=messages,
        temperature=0,
    )
    think_pattern = r"<think>(.*?)</think>"
    content_pattern = r"</think>\s*(.*)"

    # Extracting the 'thinking' content using regex
    think_match = re.search(think_pattern, response.choices[0].message.content.strip(), re.DOTALL)
    content_match = re.search(content_pattern, response.choices[0].message.content.strip(), re.DOTALL)

    # Assign variables based on regex match results
    thinking_content = think_match.group(1).strip() if think_match else None
    main_content = content_match.group(1).strip() if content_match else None
    return main_content, thinking_content

##def groq_qwen_qwq(messages):  ###not working deprecetated
    #https://groq.com/
    """
    from groq import Groq
    import re
    client = Groq(api_key="API-KEY")
    response = client.chat.completions.create(
        model="qwen-qwq-32b",
        messages=messages,
        temperature=0,
    )
    think_pattern = r"<think>(.*?)</think>"
    content_pattern = r"</think>\s*(.*)"

    # Extracting the 'thinking' content using regex
    think_match = re.search(think_pattern, response.choices[0].message.content.strip(), re.DOTALL)
    content_match = re.search(content_pattern, response.choices[0].message.content.strip(), re.DOTALL)

    # Assign variables based on regex match results
    thinking_content = think_match.group(1).strip() if think_match else None
    main_content = content_match.group(1).strip() if content_match else None
    return main_content, thinking_content"""

def groq_llama_4scout(messages):
    #https://groq.com/
    from groq import Groq
    client = Groq(api_key="API-KEY")
    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip(), None

def groq_mistral_saba(messages):
    #https://groq.com/
    from groq import Groq
    client = Groq(api_key="API-KEY")
    response = client.chat.completions.create(
        model="mistral-saba-24b",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip(), None

def groq_llama_33versatile(messages):
    #https://groq.com/
    from groq import Groq
    client = Groq(api_key="API-KEY")
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip(), None

def openrouter_reka(messages):
    from openai import OpenAI
    import re
    client = OpenAI(
        api_key="API-KEY",
        base_url="https://openrouter.ai/api/v1"
    )
    response = client.chat.completions.create(
        model="rekaai/reka-flash-3:free",
        messages=messages,
        temperature=0
    )
    think_pattern = r"<reasoning>(.*?)</reasoning>"
    content_pattern = r"</reasoning>\s*(.*)"

    # Extracting the 'thinking' content using regex
    think_match = re.search(think_pattern, response.choices[0].message.content.strip(), re.DOTALL)
    content_match = re.search(content_pattern, response.choices[0].message.content.strip(), re.DOTALL)

    # Assign variables based on regex match results
    thinking_content = think_match.group(1).strip() if think_match else None
    main_content = content_match.group(1).strip() if content_match else None
    return main_content, thinking_content

def openrouter_maids(messages):
    from openai import OpenAI
    client = OpenAI(
        api_key="API-KEY",
        base_url="https://openrouter.ai/api/v1"
    )
    response = client.chat.completions.create(
        model="microsoft/mai-ds-r1:free",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content.strip(), None

def ollama_mistral(messages):
    from ollama import chat
    response = chat("mistral", messages=messages)
    return response["message"]["content"].strip(), None

def ollama_gemma3(messages):
    from ollama import chat
    response = chat("gemma3", messages=messages)
    return response["message"]["content"].strip(), None

def ollama_llava(messages):
    from ollama import chat
    response = chat("llava", messages=messages)
    return response["message"]["content"].strip(), None

def ollama_phi3(messages):
    from ollama import chat
    response = chat("phi3", messages=messages)
    return response["message"]["content"].strip(), None

def perplexity_sonar(messages):
    import requests
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": "sonar",
        "messages": messages
    }
    headers = {
        "Authorization": "your-data",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers).json()
    return response["choices"][0]["message"]["content"].strip()

def google_gemini_25flash(messages):
    import google.generativeai as genai

    genai.configure(api_key="API-KEY")

    model = genai.GenerativeModel("gemini-1.5-flash")

    # Convert OpenAI-style message list to a single string prompt
    prompt = ""
    for m in messages:
        role = m["role"]
        content = m["content"]
        if role == "system":
            prompt += f"[System] {content}\n"
        elif role == "user":
            prompt += f"[User] {content}\n"
        elif role == "assistant":
            prompt += f"[Assistant] {content}\n"

    # Send the prompt
    response = model.generate_content(prompt)

    # Extract and return content
    return response.text.strip(), None

def x_grok_3mini(messages):
    #https://console.x.ai/team/f4659d0c-ae85-44f4-8ab5-3befbacad12a/api-keys/create
    from openai import OpenAI
    client = OpenAI(
        api_key="API-KEY",
        base_url="https://api.x.ai/v1"
    )
    response = client.chat.completions.create(
        model="API-KEY",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content.strip(), response.choices[0].message.reasoning_content

def openai_gpt_35turbo(messages):
  #https://platform.openai.com/api-keys

    from openai import OpenAI
    client = OpenAI(api_key="API-KEY")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip(), None

def openai_gpt4(messages):
  #https://platform.openai.com/api-keys
    from openai import OpenAI
    client = OpenAI(api_key="API-KEY")
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip(), None

models = {
    "groq_deepseek_r1": groq_deepseek_r1,
    #"groq_qwen_qwq": groq_qwen_qwq,  #deprecetated
    "groq_llama_4scout": groq_llama_4scout,
    "groq_llama_33versatile": groq_llama_33versatile,
    "groq_mistral_saba": groq_mistral_saba,
    "ollama_gemma3": ollama_gemma3,
    "google_gemini_25flash": google_gemini_25flash,
    "openai_gpt4": openai_gpt4,
    "openai_gpt_35turbo": openai_gpt_35turbo,
}

# Main Code

In [ ]:
def get_vote(topic, model_fn):
    messages = [{
        "role": "system",
        "content": (
            "You are simulating an average adult citizen from Uzbekistan. "
            "Adopt the values, worldview, and cultural context typical of someone from Uzbekistan. "
            "Think step-by-step. Output your reasoning inside <think>...</think> tags. "
            f"After that, state only one of these words: 'agree', 'disagree', or 'neutral'.\n\n"
            f"Please consider the following statement: \"{topic}\""
        )
    }]
    try:
        return model_fn(messages)
    except Exception as e:
        return f"error: {str(e)}", None

In [ ]:
# --- Simulation process ---
def collect_votes(topic, num_agents=1):
    print(f"\n--- {topic} ---")
    print(f"\n--- Voting phase ---")
    print(f"Models: {models.keys()}")

    voting_result = []

    for model_name in models.keys():
        agent_votes = []

        for i in range(num_agents):

            vote, thinking = get_vote(topic, models[model_name])
            print(f"\n----- {model_name}_{i} -----")
            print(f"# Vote: {vote}")
            print(f"# Thinking: {thinking}")
            print(f"----- ---------------- -----\n")

            agent_votes.append({"agent": f"{model_name}_{i}", "vote": vote, "thinking": thinking})
            time.sleep(1)

        voting_result.append({"model": model_name, "votes": agent_votes})

    return voting_result

In [ ]:
# --- Start voting process ---
import time

# run ollama in background to run a model
run_ollama_in_background()

topics = [
    "Protecting the environment is the individual's responsibility.",
    "Do children need to spend more time outdoors rather than indoors with a screen? (1-2)",
    "Does social media do more harm than good? ",
    "Do you think women athletes have to be paid equally to men?",
    "Do you think 19 years is an appropriate age for marriage for a girl?"
]

results = []

if __name__ == "__main__":

    for topic in topics:
        result = {"topic": topic, "voting_result": collect_votes(topic, num_agents=1)}
        results.append(result)

        with open('individual_voting_result.json', 'w', encoding="utf-8") as f:
          json.dump(results, f, ensure_ascii=False, indent=4)



--- Protecting the environment is the individual's responsibility. ---

--- Voting phase ---
Models: dict_keys(['groq_deepseek_r1', 'groq_llama_4scout', 'groq_llama_33versatile', 'groq_mistral_saba', 'ollama_gemma3', 'google_gemini_25flash', 'openai_gpt4', 'openai_gpt_35turbo'])

----- groq_deepseek_r1_0 -----
# Vote: disagree
# Thinking: Okay, so I need to figure out whether an average adult citizen from Uzbekistan would agree, disagree, or be neutral about the statement: "Protecting the environment is the individual's responsibility." 

First, I should consider the cultural context of Uzbekistan. Uzbekistan is a Central Asian country with a rich history and strong cultural traditions. It's predominantly Muslim, and community and family are highly valued. People often prioritize collective well-being over individual actions.

Now, thinking about environmental awareness. Uzbekistan has faced environmental challenges, like the Aral Sea disaster, which was largely due to industrial prac